The goal for running tractor is to find the ideal model of a galaxy using (a combination of) cersic, exponential, and de Vaucouleurs models. In order to do this, we need to:
- create a tractor image (which is a model of the galaxy) using the following models:
    - exponential
    - de Vaucouleurs
    - composites    
- take the model and compare it to the DECaLs model

In order to do this:
- take in the image, psf, and sig data
- create tractor image
- run the tractor optimization method


In [1]:
import os
import copy

import fitsio
import numpy as np
import pylab as plt

from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table

import sep
import tractor

from astrometry.util.fits import fits_table
#from astrometry.util.util import wcs_pv2sip_hdr

from tractor import NullWCS
from tractor.galaxy import *
from tractor.sersic import *
from tractor.psf import *
from tractor.ellipses import *

In [2]:
def fit_galaxies (galaxy_type, cat):
    #galaxy type: 
    # - "comp" = composite 
    # - "dev" = de Vaucouleurs 
    # - "exp" = exponential 
    
    sources = []
    
    if galaxy_type == 'comp':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.CompositeGalaxy(PixPos(obj['x'], obj['y']), 
                                                       Flux(0.4 * obj['flux']), GalaxyShape(obj['a'] * 0.8,  0.9,  obj['theta'] * 180.0 / np.pi),
                                                       Flux(0.6 * obj['flux']), GalaxyShape(obj['a'],  obj['b']/obj['a'], obj['theta'] * 180.0 / np.pi)))
    elif galaxy_type == 'dev':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.DevGalaxy(tractor.PixPos(obj['x'], obj['y']),
                                         tractor.Flux(obj['flux']),
                                         GalaxyShape(obj['a'] / 2.0, 
                                                     (obj['b'] / obj['a']),
                                                     (180.0 - obj['theta'] * 180.0 / np.pi))))
    elif galaxy_type == 'exp':
        for obj in obj_cat_bright:
            # if the object is a point source, add it as such
            if obj['point_source'] > 0.0:
                sources.append(tractor.PointSource(tractor.PixPos(obj['x'], obj['y']),
                                                   tractor.Flux(obj['flux'])) )
            else:
                sources.append(tractor.ExpGalaxy(tractor.PixPos(obj['x'], obj['y']),
                                                    tractor.Flux(obj['flux']),
                                                    GalaxyShape(obj['a'] / 2.0, 
                                                                (obj['b'] / obj['a']),
                                                                (180.0 - obj['theta'] * 180.0 / np.pi))))
    else: 
        raise ValueError('incorrect input for galaxy type')
        # valid types: 'comp' , 'dev' , 'exp'
                
                
    return sources         
                
    

In [4]:
directories = os.listdir()
nums = []

for directory in directories:
    #skip files
    if not str.isdigit(directory[0]):
        continue
    #open fits files
    num = directory[:3]
    if num[-1] == '_':
        num = num[:-1]
    nums.append(num)
    galaxy_dir = os.path.dirname(os.path.abspath(num+'_tractor_ready'))
    path = os.path.join(galaxy_dir,num+'_tractor_ready')

    img_data = fits.open(os.path.join(path,'img_data_crop_'+num+'.fits'))[0].data
    psf_data = fits.open(os.path.join(path,'psf_data_crop_'+num+'.fits'))[0].data
    psf_obj = tractor.PixelizedPSF(psf_data, Lorder=5)
    sig_data = fits.open(os.path.join(path,'sig_data_crop_'+num+'.fits'))[0].data
    
    print(num + ": loaded data")

    obj_cat = Table.read(path+'/tractor_'+num+'.fits')

    invvar_data = (1.0 / (sig_data ** 2.0))

    w = NullWCS(pixscale=0.168)
    
    #create tractor image
    tim = tractor.Image(data=img_data,
                invvar=invvar_data,
                psf=psf_obj,
                wcs=w,
                sky=tractor.ConstantSky(0.0),
                photocal=tractor.NullPhotoCal()
                )
    
    obj_cat_bright = copy.deepcopy(obj_cat) 
    obj_cat_bright.sort('flux')
    obj_cat_bright.reverse()

    obj_cat_bright = obj_cat_bright[:20]
    
    print(num + ": create tractor image")
    
    #prepare for model optiization
    sources_comp = fit_galaxies('comp',obj_cat_bright)
    sources_dev = fit_galaxies('dev',obj_cat_bright)
    sources_exp = fit_galaxies('exp',obj_cat_bright)
    
    trac_obj_comp = Tractor([tim], sources_comp)
    trac_mod_comp = trac_obj_comp.getModelImage(0, minsb=0.0)

    trac_obj_dev = Tractor([tim], sources_dev)
    trac_mod_dev = trac_obj_dev.getModelImage(0, minsb=0.0)

    trac_obj_exp = Tractor([tim], sources_exp)
    trac_mod_exp = trac_obj_exp.getModelImage(0, minsb=0.0)
    
    print(num + ": prepared for model optimization")
    
    # optimize de vac function
    trac_obj_dev.freezeParam('images')
    trac_obj_dev.optimize_loop()

    trac_mod_opt = trac_obj_dev.getModelImage(0, minsb=0., srcs=sources_dev)
    
    #plot images for de vac
    fig, axarr = plt.subplots(2,3)
    axarr[0,0].imshow(np.arcsinh(trac_mod_dev))
    axarr[0,0].set_title('initial model')
    axarr[0,1].imshow(np.arcsinh(img_data - trac_mod_dev))
    axarr[0,1].set_title('intial model difference')
    axarr[0,2].imshow(np.arcsinh(trac_mod_opt))
    axarr[0,2].set_title('optimized tractor model')
    axarr[1,0].imshow(np.arcsinh(img_data - trac_mod_opt))
    axarr[1,0].set_title('optimized model difference')
    axarr[1,1].imshow(np.arcsinh(img_data))
    axarr[1,1].set_title('initial image')
    axarr[1,2].axis('off')

    fig.tight_layout(rect=[0, 0, 1.5, 1.5])
    plt.subplots_adjust(wspace=0, hspace=.2)
    plt.suptitle("De Vaucouleurs model",x=.75,y=1.6,fontsize=20)

    axes = [axarr[0,0],axarr[0,1],axarr[0,2],axarr[1,0],axarr[1,1],axarr[1,2]]
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.savefig(num+'_dev.png', bbox_inches='tight')
    
    print(num + ": devac image created")
    
    # exp function
    trac_obj_exp.freezeParam('images')
    trac_obj_exp.optimize_loop()

    trac_mod_opt = trac_obj_exp.getModelImage(0, minsb=0., srcs=sources_exp)
    
    fig, axarr = plt.subplots(2,3)
    axarr[0,0].imshow(np.arcsinh(trac_mod_exp))
    axarr[0,0].set_title('initial model')
    axarr[0,1].imshow(np.arcsinh(img_data - trac_mod_exp))
    axarr[0,1].set_title('intial model difference')
    axarr[0,2].imshow(np.arcsinh(trac_mod_opt))
    axarr[0,2].set_title('optimized tractor model')
    axarr[1,0].imshow(np.arcsinh(img_data - trac_mod_opt))
    axarr[1,0].set_title('optimized model difference')
    axarr[1,1].imshow(np.arcsinh(img_data))
    axarr[1,1].set_title('initial image')
    axarr[1,2].axis('off')

    fig.tight_layout(rect=[0, 0, 1.5, 1.5])
    plt.subplots_adjust(wspace=0, hspace=.2)
    plt.suptitle("Exponential model",x=.75,y=1.6,fontsize=20)

    axes = [axarr[0,0],axarr[0,1],axarr[0,2],axarr[1,0],axarr[1,1],axarr[1,2]]
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])
        
    plt.savefig(num+'_exp.png', bbox_inches='tight')
    
    print(num + ": exp image created")
        
    # composite function
    trac_obj_comp.freezeParam('images')
    trac_obj_comp.optimize_loop()

    trac_mod_opt = trac_obj_comp.getModelImage(0, minsb=0., srcs=sources_comp)
    
    fig, axarr = plt.subplots(2,3)
    axarr[0,0].imshow(np.arcsinh(trac_mod_comp))
    axarr[0,0].set_title('initial model')
    axarr[0,1].imshow(np.arcsinh(img_data - trac_mod_comp))
    axarr[0,1].set_title('intial model difference')
    axarr[0,2].imshow(np.arcsinh(trac_mod_opt))
    axarr[0,2].set_title('optimized tractor model')
    axarr[1,0].imshow(np.arcsinh(img_data - trac_mod_opt))
    axarr[1,0].set_title('optimized model difference')
    axarr[1,1].imshow(np.arcsinh(img_data))
    axarr[1,1].set_title('initial image')
    axarr[1,2].axis('off')

    fig.tight_layout(rect=[0, 0, 1.5, 1.5])
    plt.subplots_adjust(wspace=0, hspace=.2)
    plt.suptitle("Composite model",x=.75,y=1.6,fontsize=20)

    axes = [axarr[0,0],axarr[0,1],axarr[0,2],axarr[1,0],axarr[1,1],axarr[1,2]]
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])
    plt.savefig(num+'_comp.png', bbox_inches='tight')
        
    
    print(num + ": comp image created")
    

126: loaded data
126: create tractor image
126: prepared for model optimization
patchx is None:
   DevGalaxy at pixel (8.99, 406.19) with Flux: 846.017 and Galaxy Shape: re=-5.00, ab=0.67, phi=298.8
  stepping galaxy shape re
  stepped 1.0
  to -5.00019541257
patchx is None:
   DevGalaxy at pixel (8.99, 406.19) with Flux: 846.017 and Galaxy Shape: re=-5.00, ab=0.67, phi=298.8
  stepping galaxy shape ab
  stepped 1.0
  to 0.665252876821
patchx is None:
   DevGalaxy at pixel (8.99, 406.19) with Flux: 846.017 and Galaxy Shape: re=-5.00, ab=0.67, phi=298.8
  stepping galaxy shape phi
  stepped 1.0
  to 298.791494523
patchx is None:
   DevGalaxy at pixel (8.99, 406.19) with Flux: 30821.5 and Galaxy Shape: re=-5.00, ab=0.67, phi=298.8
  stepping galaxy shape re
  stepped 1.0
  to -5.00019541257
patchx is None:
   DevGalaxy at pixel (8.99, 406.19) with Flux: 30821.5 and Galaxy Shape: re=-5.00, ab=0.67, phi=298.8
  stepping galaxy shape ab
  stepped 1.0
  to 0.665252876821
patchx is None:
   D

patchx is None:
   ExpGalaxy at pixel (4.32, 407.30) with Flux: 33988.4 and Galaxy Shape: re=-0.38, ab=1.82, phi=212.9
  stepping galaxy shape re
  stepped 1.0
  to -0.379290031956
patchx is None:
   ExpGalaxy at pixel (4.32, 407.30) with Flux: 33988.4 and Galaxy Shape: re=-0.38, ab=1.82, phi=212.9
  stepping galaxy shape ab
  stepped 1.0
  to 1.8165451405
patchx is None:
   ExpGalaxy at pixel (4.32, 407.30) with Flux: 33988.4 and Galaxy Shape: re=-0.38, ab=1.82, phi=212.9
  stepping galaxy shape phi
  stepped 1.0
  to 212.861069448
patchx is None:
   ExpGalaxy at pixel (-374487.82, 372556.79) with Flux: 2007.17 and Galaxy Shape: re=22739.61, ab=13393.68, phi=-12909.8
  stepping galaxy shape re
  stepped 1.0
  to 22739.6109272
patchx is None:
   ExpGalaxy at pixel (-374487.82, 372556.79) with Flux: 2007.17 and Galaxy Shape: re=22739.61, ab=13393.68, phi=-12909.8
  stepping galaxy shape ab
  stepped 1.0
  to 13393.6796602
patchx is None:
   ExpGalaxy at pixel (-374487.82, 372556.79) wit

IndexError: index 102 is out of bounds for axis 1 with size 102

In [ ]:
import shutil
#organize files

for num in nums:
    os.makedirs(num+'_tractor')
    files = [num+'_exp.png',num+'_comp.png',num+'_dev.png']

    for file in files: 
        src = os.path.abspath(file)
        dst = os.path.abspath(num+'_tractor')
        shutil.move(src,dst)
